In [1]:
%cd ..

/home/olle/Documents/OrderBookRL


In [2]:
%load_ext line_profiler

In [3]:
%load_ext cython

In [4]:
import ujson
import pickle
import os
import math
import numpy as np
import pandas as pd
import pickle
import feather

In [5]:
import ujson
import pickle
import os
import msgpack
import math

files = os.listdir('data/json/')
snap_files = sorted([filename for filename in files if 'snaps' in filename])
    
for snapfile in snap_files:
    with open('data/json/' + snapfile , 'r') as f:
        snaps = f.readlines()
        for snap in snaps:
            snap = ujson.loads(snap)
            seq = snap['sequence']
            print(seq)
            with open('data/snap_json/snap_' + str(seq) + '.json', 'w') as snapf:
                ujson.dump(snap, snapf)

5486159990
5486593951
5487047820
5487521543
5487964732
5488346034
5488728583
5489091550
5489439697
5489848136
5490234651
5490586963
5490952864
5491359778
5491790864


In [6]:
files = os.listdir('data/json/')
mess_files = sorted([filename for filename in files if 'mess' in filename])


keys = set(['order_type', 'reason','sequence','side','size','type', 'price', 'funds', 'order_id', 'time'])
cat_cols = ['order_type', 'reason', 'side', 'type']
price_tick = 0.01
price_dec = int(np.log10(1/price_tick))

#mess_files = mess_files[0:2]
for k, messfile in enumerate(mess_files):
    
    print(k)
    messages = []
    with open('data/json/' + messfile , 'r') as f:
        mess = f.readlines()
        for m in mess:
            ms = ujson.loads(m)
            ms = {k:v for k,v in ms.items() if k in keys}
            messages.append(ms)
    
    df = pd.DataFrame(messages)
    
    df['funds'] = df['funds'].astype(float)
    df['price'] = df['price'].astype(float).round(price_dec)
    df['size'] = df['size'].astype(float)
    df.replace('sell', 1, inplace=True)
    df.replace('buy', 0, inplace=True)
    df.side = df.side.fillna(-1)
    df.side = df.side.astype(int)
    df['trader_id'] = -1
    #df.time = pd.to_datetime(df.time)
    df.loc[df['size'].isnull(), 'size'] = -1
    feather.write_dataframe(df, 'data/feather/' + str(k) + '.feather')

0
1
2
3
4
5


In [7]:
df.head()

,funds,order_id,order_type,price,reason,sequence,side,size,time,type,trader_id
0,NaN,cdd8dcb6-015d-4cc2-b684-a8a9e21db517,NaN,8930.02,canceled,5491060906,0,-1.0,2018-03-24T21:00:02.567000Z,done,-1
1,NaN,87c33f1b-b88b-40ca-9249-e3900f0ab9d5,limit,8930.03,NaN,5491060907,0,0.9,2018-03-24T21:00:02.582000Z,received,-1
2,NaN,87c33f1b-b88b-40ca-9249-e3900f0ab9d5,NaN,8930.03,NaN,5491060908,0,-1.0,2018-03-24T21:00:02.582000Z,open,-1
3,NaN,c618aba3-73f8-4d5c-8f4f-97b23e52ee07,NaN,8953.29,canceled,5491060909,1,-1.0,2018-03-24T21:00:02.582000Z,done,-1
4,NaN,87c33f1b-b88b-40ca-9249-e3900f0ab9d5,NaN,8930.03,canceled,5491060910,0,-1.0,2018-03-24T21:00:02.588000Z,done,-1
